In [113]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor

In [114]:
population = pd.read_csv(r"C:\Users\sathv\OneDrive\Desktop\RDM\adult.data.csv")

In [115]:
sample_1 = population

In [116]:
X= sample_1[["age","workclass","education-num","marital-status","occupation","relationship","race","sex","hoursperweek","native-country"]]
y = sample_1["class"]

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [118]:
X_train

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hoursperweek,native-country
5514,33,Local-gov,13,Never-married,Prof-specialty,Not-in-family,White,Female,50,United-States
19777,36,Private,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States
10781,58,Self-emp-not-inc,5,Separated,Craft-repair,Not-in-family,White,Male,40,United-States
32240,21,Private,11,Married-civ-spouse,Farming-fishing,Husband,White,Male,46,United-States
9876,27,Private,10,Divorced,Other-service,Not-in-family,White,Male,40,United-States
...,...,...,...,...,...,...,...,...,...,...
29802,47,Private,13,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States
5390,31,Private,8,Divorced,Other-service,Unmarried,White,Female,21,United-States
860,18,Private,7,Never-married,Adm-clerical,Own-child,White,Female,20,United-States
15795,50,Self-emp-not-inc,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,84,United-States


In [119]:
def detect_outlier(data,col):
    only_na = []
    new_data = data.copy()
    clf = LocalOutlierFactor(n_neighbors=30)
    normalized_df = (new_data[col]- new_data[col].mean())/new_data[col].std()
    clf.fit_predict(normalized_df.values.reshape(-1,1))
    results = clf.negative_outlier_factor_
    new_data["LOF"] = results.tolist()
    new_data = new_data.query("LOF < -1.5")
    for i in new_data.index:
        only_na.append(i)
    return only_na

In [120]:
def impute(data,col,index):
    new_data = data.copy()
    mode = new_data[col].mode()
    for i in index:
        new_data.at[i, col]=mode
    return new_data
        

In [121]:
def drop_rows(x,only_na):
    new_data = x.copy()
    for i in only_na:
        new_data.drop(i,inplace=True)

    return new_data

In [122]:
index= detect_outlier(X_train,"age")

In [123]:
X_train = impute(X_train,"age",index)

In [124]:
index = detect_outlier(X_train,"hoursperweek")

In [125]:
X_train = impute(X_train,"hoursperweek",index)

In [126]:
X_train = pd.get_dummies(X_train, columns = ['workclass', 'relationship', 'marital-status', 'occupation', 'race', 'sex', 'native-country'])

In [127]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26048 entries, 5514 to 23654
Data columns (total 89 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   age                                         26048 non-null  int64
 1   education-num                               26048 non-null  int64
 2   hoursperweek                                26048 non-null  int64
 3   workclass_ ?                                26048 non-null  uint8
 4   workclass_ Federal-gov                      26048 non-null  uint8
 5   workclass_ Local-gov                        26048 non-null  uint8
 6   workclass_ Never-worked                     26048 non-null  uint8
 7   workclass_ Private                          26048 non-null  uint8
 8   workclass_ Self-emp-inc                     26048 non-null  uint8
 9   workclass_ Self-emp-not-inc                 26048 non-null  uint8
 10  workclass_ State-gov           

In [128]:
y_train = y_train.astype('category')

In [129]:
X_test = pd.get_dummies(X_test, columns = ['workclass', 'relationship', 'marital-status', 'occupation', 'race', 'sex', 'native-country'])

In [130]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6513 entries, 14160 to 24639
Data columns (total 86 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   age                                    6513 non-null   int64
 1   education-num                          6513 non-null   int64
 2   hoursperweek                           6513 non-null   int64
 3   workclass_ ?                           6513 non-null   uint8
 4   workclass_ Federal-gov                 6513 non-null   uint8
 5   workclass_ Local-gov                   6513 non-null   uint8
 6   workclass_ Private                     6513 non-null   uint8
 7   workclass_ Self-emp-inc                6513 non-null   uint8
 8   workclass_ Self-emp-not-inc            6513 non-null   uint8
 9   workclass_ State-gov                   6513 non-null   uint8
 10  workclass_ Without-pay                 6513 non-null   uint8
 11  relationship_ Husband    

In [131]:
X_train.columns.difference(X_test.columns)

Index(['native-country_ Holand-Netherlands',
       'native-country_ Outlying-US(Guam-USVI-etc)',
       'workclass_ Never-worked'],
      dtype='object')

In [132]:
X_train["workclass_ Never-worked"]

5514     0
19777    0
10781    0
32240    0
9876     0
        ..
29802    0
5390     0
860      0
15795    0
23654    0
Name: workclass_ Never-worked, Length: 26048, dtype: uint8

In [133]:
# X_test["workclass_Never-worked "].drop
X_train = X_train.drop(['workclass_ Never-worked'], axis=1)

In [134]:
X_train = X_train.drop(['native-country_ Holand-Netherlands'], axis=1)

In [135]:
X_train = X_train.drop(['native-country_ Outlying-US(Guam-USVI-etc)'], axis=1)

In [136]:
y_test = y_test.astype('category')

In [137]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
logisticRegr = LogisticRegression()

In [138]:
def logistic_regression_train(logisticRegr,x,y):
    logisticRegr.fit(x,y)
def logistic_regression_test(logisticRegr,x):
    y_pred = logisticRegr.predict_proba(x)
    return y_pred
def logistic_regression_pred_test(logisticRegr,x):
    y_pred_test = logisticRegr.predict(x)
    return y_pred_test

def average_diff(y_pred, X_test):
    column_i = y_pred.T[0]
    X_test_copy = X_test.copy()
    X_test_copy['preds'] = column_i
    Female = X_test_copy["preds"][X_test_copy["sex_ Female"]== 1]
    female_average = sum(Female)/len(Female)
    male = X_test_copy["preds"][X_test_copy["sex_ Male"] == 1]
    male_average = sum(male)/len(male)
    diff = (female_average - male_average)
    return diff 

def accuracy(y_test, y_pred_final):
#     y_pred_final = np.argmax(y_pred, axis=1)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_final).ravel()
    true_positive  = tn
    false_positive = fn
    true_negative  = tp
    false_negative = fp
    accuracy = (true_positive + true_negative) / (true_negative + true_positive + false_positive + false_negative )
    return(accuracy)
    
    

In [139]:
model = logistic_regression_train(logisticRegr, X_train, y_train)

C:\Users\sathv\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [140]:
y_pred = logistic_regression_test(logisticRegr, X_test)

In [141]:
y_pred

array([[0.97174508, 0.02825492],
       [0.46430759, 0.53569241],
       [0.22980319, 0.77019681],
       ...,
       [0.13975102, 0.86024898],
       [0.95560919, 0.04439081],
       [0.99385381, 0.00614619]])

In [142]:
dirty_diff = average_diff(y_pred, X_test)

In [143]:
dirty_diff

0.18883654232056923

In [144]:
dirty_test_score = logisticRegr.score(X_test, y_test)

In [145]:
dirty_test_score

0.8334101028711807

In [146]:
X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [147]:
index = detect_outlier(X_clean_train,"age")
X_clean_train = impute(X_clean_train,"age",index)
index = detect_outlier(X_clean_train,"hoursperweek")
X_clean_train = impute(X_clean_train,"hoursperweek",index)

In [148]:
index= detect_outlier(X_clean_test,"age")
X_clean_test = impute(X_clean_test,"age",index)
index = detect_outlier(X_clean_test,"hoursperweek")
X_clean_test = impute(X_clean_test,"hoursperweek",index)

In [149]:
X_clean_train = pd.get_dummies(X_clean_train, columns = ['workclass', 'relationship', 'marital-status', 'occupation', 'race', 'sex', 'native-country'])

In [150]:
y_clean_train = y_clean_train.astype('category')

In [151]:
X_clean_test = pd.get_dummies(X_clean_test, columns = ['workclass', 'relationship', 'marital-status', 'occupation', 'race', 'sex', 'native-country'])

In [152]:
y_clean_test = y_clean_test.astype('category')

In [153]:
X_clean_train.columns.difference(X_clean_test.columns)

Index(['native-country_ Holand-Netherlands',
       'native-country_ Outlying-US(Guam-USVI-etc)',
       'workclass_ Never-worked'],
      dtype='object')

In [154]:
X_clean_train = X_clean_train.drop(['native-country_ Holand-Netherlands','native-country_ Outlying-US(Guam-USVI-etc)','workclass_ Never-worked' ], axis=1)

In [155]:
clean_model = logistic_regression_train(logisticRegr, X_clean_train, y_clean_train)

C:\Users\sathv\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [156]:
y_clean_pred = logistic_regression_test(logisticRegr, X_clean_test)

In [157]:
y_clean_pred

array([[0.97174508, 0.02825492],
       [0.46430759, 0.53569241],
       [0.22980319, 0.77019681],
       ...,
       [0.13975102, 0.86024898],
       [0.95560919, 0.04439081],
       [0.99385381, 0.00614619]])

In [158]:
clean_diff = average_diff(y_clean_pred, X_clean_test)

In [159]:
clean_diff

0.1865727948664242

In [160]:
clean_test_score = logisticRegr.score(X_clean_test, y_clean_test)

In [161]:
clean_test_score

0.8334101028711807

In [162]:
from tabulate import tabulate
data = [["Local Outlier Factor","Impute by mode","Dirty test data","One-Hot Encoding", "Logistic Regression", dirty_test_score, dirty_diff],
["Local Outlier Factor","Impute by mode", "Clean test data", "One-Hot Encoding", "Logistic Regression", clean_test_score, clean_diff]]
print (tabulate(data, headers=["Detect Method","Method", "Test Data", "Encoding Technique", "ML Model", "Accuracy", "Fairness"]))

Detect Method         Method          Test Data        Encoding Technique    ML Model               Accuracy    Fairness
--------------------  --------------  ---------------  --------------------  -------------------  ----------  ----------
Local Outlier Factor  Impute by mode  Dirty test data  One-Hot Encoding      Logistic Regression     0.83341    0.188837
Local Outlier Factor  Impute by mode  Clean test data  One-Hot Encoding      Logistic Regression     0.83341    0.186573
